In [1]:
#for dev purpose
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [2]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta
from snp_query_box import DsnpHelperFunction, snpQueries
from snp_query_box.dsnp_transform import dsnpTransform

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating pull_ctm_bq and pull_grievance_bq in the shared storage")

User: a845514 Started creating pull_ctm_bq and pull_grievance_bq in the shared storage


In [3]:
# set the starting time
start_time = time.time()

In [4]:
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)

last_month = last_date_of_last_month_str[:7]
one_year_rolling_back_date = pd.Timestamp(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1))
one_year_rolling_back_date_str = one_year_rolling_back_date.strftime('%Y-%m-%d')
print(one_year_rolling_back_date)

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print(first_date_of_last_month_str)
reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)


2024-06-01 00:00:00
2025-01-01
2025-05-01
2025-05-31


In [5]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files/2025-05-31
The folder already exist
Output files will sit in the folder above.


In [6]:
#locate data dump folder
if new_pull:
    pull_ctm_bq = snpQueries.pull_ctm_bq(start_date=first_date_of_year, end_date=reporting_end_date)
    pull_ctm_bq.to_parquet(f"{output_path}/pull_ctm_bq.parquet")
else:
    try:
        pull_ctm_bq = pd.read_parquet(f"{output_path}/pull_ctm_bq.parquet")

    except:
        pull_ctm_bq = snpQueries.pull_ctm_bq(start_date=first_date_of_year, end_date=reporting_end_date)
        pull_ctm_bq.to_parquet(f"{output_path}/pull_ctm_bq.parquet")

In [7]:
#locate data dump folder
if new_pull:
    pull_grievance_bq = snpQueries.pull_grievance_bq(start_date=first_date_of_year, end_date=reporting_end_date)
    pull_grievance_bq.to_parquet(f"{output_path}/pull_grievance_bq.parquet")
else:
    try:
        pull_grievance_bq = pd.read_parquet(f"{output_path}/pull_grievance_bq.parquet")

    except:
        pull_grievance_bq = snpQueries.pull_grievance_bq(start_date=first_date_of_year, end_date=reporting_end_date)
        pull_grievance_bq.to_parquet(f"{output_path}/pull_grievance_bq.parquet")

In [8]:
print(pull_grievance_bq.shape)
print(pull_grievance_bq.head())

(137677, 15)
        GRV_NBR Contract_Number  PBP LIS_Flag MEDICARE_NUMBER SOURCE   
0  G25056923807           H3288  002     None     2TJ1A83XJ33  Phone  \
1  G25107781629           H7301  013     None     8TN0U29EK17  Phone   
2  G25136380720           H5521  469        1     2YC4K31PF93  Phone   
3  G25023944066           H5522  803     None     6KV0VD3JJ22  Phone   
4  G25144864758           H5521  425     None     5X32EE9UE25  Phone   

               CTG                       SUB_CTG  PRIORITY   
0     Claims Issue       Incorrect Provider Type  Standard  \
1          Billing  Account Adjustments Not Made  Standard   
2  Quality of Care                           DME  Standard   
3        Marketing  High pressure sales tactics   Standard   
4     Claims Issue        Claim Processed as OON  Standard   

                  OCCUR_DTS              REPORTED_DTS  REPORTING_YEAR   
0 2025-02-24 00:00:00+00:00 2025-02-24 00:00:00+00:00            2025  \
1 2025-04-16 09:26:00+00:00 2025-04

In [9]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))

Process time -- 48.20893692970276 seconds
